In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import newton, bisect
from scipy.differentiate import derivative

In [3]:
f = lambda x: x**2 - 3*x + 2

fi1 = lambda x: (x**2 + 2) / 3
fi2 = lambda x: np.sqrt(3*x - 2)
fi3 = lambda x: 3 - (2 / x)
fi4 = lambda x: (x**2 - 2) / (2*x - 3)

dfi1_2 = derivative(fi1, 2)
dfi2_2 = derivative(fi2, 2)
dfi3_2 = derivative(fi3, 2)
dfi4_2 = derivative(fi4, 2)
[dfi1_2.df, dfi2_2.df, dfi3_2.df, dfi4_2.df]

C:\Users\Szymon\AppData\Local\Temp\ipykernel_16032\276319913.py:6: RuntimeWarning: divide by zero encountered in divide
  fi4 = lambda x: (x**2 - 2) / (2*x - 3)


[np.float64(1.3333333333333286),
 np.float64(0.7499999999936833),
 np.float64(0.4999999999925677),
 np.float64(nan)]